In [1]:
pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import requests
import pandas as pd
from pandas.core.internals.array_manager import NullArrayProxy
import time
import datetime

get races info

In [50]:
races = {'season': [],
        'round': [],
        'circuit_id': [],
        'lat': [],
        'long': [],
        'country': [],
        'city':[],
        'date': [],
        'sprint' : []}

for year in list(range(2014,2023)):
    
    url = 'https://ergast.com/api/f1/{}.json'
    r = requests.get(url.format(year))
    json = r.json()

    for item in json['MRData']['RaceTable']['Races']:
        try:
            races['season'].append(int(item['season']))
        except:
            races['season'].append(None)

        try:
            races['round'].append(int(item['round']))
        except:
            races['round'].append(None)

        try:
            races['circuit_id'].append(item['Circuit']['circuitId'])
        except:
            races['circuit_id'].append(None)

        try:
            races['lat'].append(float(item['Circuit']['Location']['lat']))
        except:
            races['lat'].append(None)

        try:
            races['long'].append(float(item['Circuit']['Location']['long']))
        except:
            races['long'].append(None)

        try:
            races['country'].append(item['Circuit']['Location']['country'])
        except:
            races['country'].append(None)

        try:
            races['city'].append(item['Circuit']['Location']['locality'])
        except:
            races['city'].append(None)

        try:
            races['date'].append(item['date'])
        except:
            races['date'].append(None)

        try:
            item['Sprint']['date'] is not None
        except:
            races['sprint'].append(0)
        else:
            races['sprint'].append(1)
        
races = pd.DataFrame(races)
print(races.shape)

(182, 9)


get races results

In [51]:
for index, row in races.iterrows():
    url = 'https://ergast.com/api/f1/{}/{}/results/1.json'.format(row['season'],row['round'])
    r = requests.get(url)
    json = r.json()

    for race in json['MRData']['RaceTable']['Races']:
      if race['Results'][0]['Driver']['givenName']:
        races.loc[index, 'driver'] = race['Results'][0]['Driver']['givenName']+' '+race['Results'][0]['Driver']['familyName']
      else:
        races.loc[index, 'driver'] = None

get driver table

In [52]:
drivers = {'driver_id': [],
            'driver' : []}

url = 'https://ergast.com/api/f1/2022/drivers.json'
r = requests.get(url)
json = r.json()
num = 0

for item in json['MRData']['DriverTable']['Drivers']:
    try:
      drivers['driver'].append(item['givenName']+' '+item['familyName'])
      drivers['driver_id'].append(num)
      num += 1
    except:
      drivers['driver'].append(None)
      drivers['driver_id'].append(None)

drivers = pd.DataFrame(drivers)

In [6]:
drivers.to_csv('drivers.csv') 

In [53]:
races = pd.merge(races, drivers, on=['driver'], how='left')

In [54]:
races = races.drop(['driver'], axis=1)

In [55]:
races = races.dropna()

In [56]:
races = races.rename({'driver_id':'position_1'}, axis='columns')

get qualifying results

In [57]:
for index, row in races.iterrows():
    url = 'https://ergast.com/api/f1/{}/{}/qualifying/1.json'.format(row['season'],row['round'])
    r = requests.get(url)
    json = r.json()

    for item in json['MRData']['RaceTable']['Races']:
      for race in item['QualifyingResults']:
        if race['Driver']['givenName']:
          races.loc[index, 'driver'] = race['Driver']['givenName']+' '+race['Driver']['familyName']

        else:
          races.loc[index, 'driver'] = None

In [58]:
races = pd.merge(races, drivers, on=['driver'], how='left')

In [59]:
races = races.drop(['driver'], axis=1)

In [60]:
races = races.rename({'driver_id':'q_p1'}, axis='columns')

In [61]:
races = races.fillna(0)

In [62]:
for index, row in races.iterrows():
    url = 'https://ergast.com/api/f1/{}/{}/qualifying/2.json'.format(row['season'],row['round'])
    r = requests.get(url)
    json = r.json()

    for item in json['MRData']['RaceTable']['Races']:
      for race in item['QualifyingResults']:
        if race['Driver']['givenName']:
          races.loc[index, 'driver'] = race['Driver']['givenName']+' '+race['Driver']['familyName']

        else:
          races.loc[index, 'driver'] = None

In [63]:
races = pd.merge(races, drivers, on=['driver'], how='left')

In [64]:
races = races.drop(['driver'], axis=1)

In [65]:
races = races.rename({'driver_id':'q_p2'}, axis='columns')

In [66]:
races = races.fillna(0)

In [67]:
for index, row in races.iterrows():
    url = 'https://ergast.com/api/f1/{}/{}/qualifying/3.json'.format(row['season'],row['round'])
    r = requests.get(url)
    json = r.json()

    for item in json['MRData']['RaceTable']['Races']:
      for race in item['QualifyingResults']:
        if race['Driver']['givenName']:
          races.loc[index, 'driver'] = race['Driver']['givenName']+' '+race['Driver']['familyName']

        else:
          races.loc[index, 'driver'] = None

In [68]:
races = pd.merge(races, drivers, on=['driver'], how='left')

In [69]:
races = races.drop(['driver'], axis=1)

In [70]:
races = races.rename({'driver_id':'q_p3'}, axis='columns')

In [71]:
races = races.fillna(0)

get sprint qualifying results

In [25]:
for index, row in races.iterrows():
    if row['sprint'] == 1:
        url = 'https://ergast.com/api/f1/{}/{}/sprint/1.json'.format(row['season'],row['round'])
        r = requests.get(url)
        json = r.json()

        for item in json['MRData']['RaceTable']['Races']:
          for race in item['SprintResults']:
            if race['Driver']['givenName']:
              races.loc[index, 'driver'] = race['Driver']['givenName']+' '+race['Driver']['familyName']
            else:
              races.loc[index, 'driver'] = None

In [26]:
races = pd.merge(races, drivers, on=['driver'], how='left')

In [27]:
races = races.drop(['driver'], axis=1)

In [28]:
races = races.rename({'driver_id':'sprint_p1'}, axis='columns')

In [29]:
races['sprint_p1'] = races['sprint_p1'].fillna(0) 

get weather for racing date

In [72]:
for index, row in races.iterrows():
  url = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{}%2C{}/{}?unitGroup=metric&elements=temp%2Cwindspeed%2Cconditions&include=days&key=UHADRGQA7EE52B4HAATFN5A2Q&contentType=json'.format(row['lat'],row['long'],row['date'])
  r = requests.get(url)
  json = r.json()
  for item in json['days']:
        try:
            races.loc[index, 'temp'] = item['temp']
        except:
            races.loc[index, 'temp'] = None

        try:
            races.loc[index, 'windspeed'] = item['windspeed']
        except:
            races.loc[index, 'windspeed'] = None

        try:
            races.loc[index, 'conditions'] = item['conditions']
        except:
            races.loc[index, 'conditions'] = None


In [73]:
races['conditions'] = races['conditions'].str.split(', ')

In [74]:
for index, row  in races.iterrows():
  if row['conditions'][0] != 'Rain':
    races.loc[index, 'rain'] = 0
    races.loc[index, 'weather'] = row['conditions'][0]
  else:
    races.loc[index, 'rain'] = 1
    try:
      races.loc[index, 'weather'] = row['conditions'][1]
    except:
      races.loc[index, 'weather'] = 'Precipitation'

transform data

In [75]:
from sklearn.preprocessing import OneHotEncoder

ohe_weather = OneHotEncoder()
transformed_weather = ohe_weather.fit_transform(races[['weather']])

In [76]:
ohe_weather.categories_[0]

array(['Clear', 'Overcast', 'Partially cloudy', 'Precipitation'],
      dtype=object)

In [77]:
races[['Clear', 'Overcast', 'Partially cloudy', 'Precipitation']] = transformed_weather.toarray()

In [78]:
lambda x: x*10 if x<2 else (x**2 if x<4 else x+10)

<function __main__.<lambda>>

In [79]:
races['wind'] = [1 if x<12 else (2 if x>=12 and x<20 else (3 if x>=20 and x<30 else(4 if x>=30 and x<40 else 5))) for x in races['windspeed']]

In [80]:
ohe_circutid = OneHotEncoder()
transformed_circutid = ohe_circutid.fit_transform(races[['circuit_id']])

In [81]:
ohe_circutid.categories_[0]

array(['albert_park', 'americas', 'bahrain', 'baku', 'catalunya',
       'hockenheimring', 'hungaroring', 'imola', 'interlagos', 'istanbul',
       'jeddah', 'losail', 'marina_bay', 'miami', 'monaco', 'monza',
       'mugello', 'nurburgring', 'portimao', 'red_bull_ring', 'ricard',
       'rodriguez', 'sepang', 'shanghai', 'silverstone', 'sochi', 'spa',
       'suzuka', 'villeneuve', 'yas_marina', 'zandvoort'], dtype=object)

In [82]:
races[['albert_park', 'americas', 'bahrain', 'baku', 'catalunya',
       'hockenheimring', 'hungaroring', 'imola', 'interlagos', 'istanbul',
       'jeddah', 'losail', 'marina_bay', 'miami', 'monaco', 'monza',
       'mugello', 'nurburgring', 'portimao', 'red_bull_ring', 'ricard',
       'rodriguez', 'sepang', 'shanghai', 'silverstone', 'sochi', 'spa',
       'suzuka', 'villeneuve', 'yas_marina', 'zandvoort']] = transformed_circutid.toarray()

In [83]:
races = races.drop(['circuit_id', 'lat', 'long', 'country', 'city', 'date', 'windspeed', 'conditions', 'weather'], axis=1)

In [84]:
races.columns

Index(['season', 'round', 'sprint', 'position_1', 'q_p1', 'q_p2', 'q_p3',
       'temp', 'rain', 'Clear', 'Overcast', 'Partially cloudy',
       'Precipitation', 'wind', 'albert_park', 'americas', 'bahrain', 'baku',
       'catalunya', 'hockenheimring', 'hungaroring', 'imola', 'interlagos',
       'istanbul', 'jeddah', 'losail', 'marina_bay', 'miami', 'monaco',
       'monza', 'mugello', 'nurburgring', 'portimao', 'red_bull_ring',
       'ricard', 'rodriguez', 'sepang', 'shanghai', 'silverstone', 'sochi',
       'spa', 'suzuka', 'villeneuve', 'yas_marina', 'zandvoort'],
      dtype='object')

In [85]:
races = races.sort_values(['season', 'round'])

In [86]:
races

,season,round,sprint,position_1,q_p1,q_p2,q_p3,temp,rain,Clear,...,rodriguez,sepang,shanghai,silverstone,sochi,spa,suzuka,villeneuve,yas_marina,zandvoort
0,2014,2,0,4.0,4.0,19.0,0.0,28.1,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2014,3,0,4.0,0.0,4.0,12.0,23.1,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2014,4,0,4.0,4.0,12.0,19.0,14.9,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2014,5,0,4.0,4.0,0.0,12.0,19.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2014,7,0,12.0,0.0,4.0,19.0,22.2,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,2022,9,0,18.0,18.0,1.0,14.0,15.8,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
148,2022,10,0,14.0,14.0,18.0,7.0,14.5,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
149,2022,11,1,7.0,18.0,7.0,14.0,16.2,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
150,2022,12,0,18.0,7.0,18.0,11.0,27.7,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [87]:
races.to_csv('races.csv') 